# ABS Quarterly Labour Account 6150

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# local imports
from plotting import (
    set_chart_dir,
    recalibrate_series,
    clear_chart_dir,
    plot_covid_recovery,
    line_plot,
    plot_growth_finalise,
    calc_growth,
)
from abs_data_capture import (
    get_fs_constants,
    AbsLandingPage,
    get_abs_data,
    get_plot_constants,
    metacol,
    get_identifier,
)

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
SHOW = False

## Get data from ABS

In [4]:
landing_page = AbsLandingPage(
    theme="labour", parent_topic="labour-accounts", topic="labour-account-australia"
)
abs_labour = get_abs_data(landing_page)
source, CHART_DIR, labour_catalogue_id, meta_labour = get_fs_constants(
    abs_labour, landing_page
)
RECENT, plot_times, plot_tags = get_plot_constants(meta_labour)

Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/b6880764e99e5f4aca443b59f4c19a3d--All-time-series-spreadsheets.zip
Extracting DataFrames from the zip-file ...


In [5]:
landing_page = AbsLandingPage(
    theme="economy",
    parent_topic="national-accounts",
    topic="australian-national-accounts-national-income-expenditure-and-product",
)
abs_gdp = get_abs_data(landing_page)
_, _, gdp_catalogue_id, meta_gdp = get_fs_constants(abs_gdp, landing_page)

Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/5017be8287dea982b865689c39593eff--All_time_series_workbooks.zip
Extracting DataFrames from the zip-file ...


## Plot

In [6]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Labour productivity - GDP per hour worked ...

In [7]:
# GDP ...
gdp_table = "1"
series_type = "Seasonally Adjusted"
did = "Gross domestic product: Chain volume measures ;"
gdp_id, units = get_identifier(meta_gdp, did, series_type, gdp_table)
gdp = abs_gdp[gdp_table][gdp_id] * 1_000_000

# Hours worked ...
labour_table = "1"
series_type = "Seasonally Adjusted"
did = "Volume; Labour Account hours actually worked in all jobs ;  Australia ;  Total all industries ;"
hours_id, units = get_identifier(meta_labour, did, series_type, labour_table)
hours = abs_labour[labour_table][hours_id] * 1_000

# calculate productivity and plot
productivity = gdp / hours
ax = productivity.dropna().plot(lw=2.5)
title = "Labour Productivity: GDP per Hour Worked"
lfooter = (
    f"Australia. {series_type.capitalize()}. GDP: Chain volume measures. "
    "Hours: Total actual hours worked. "
)
rfooter = f"ABS {gdp_catalogue_id}:{gdp_table} {labour_catalogue_id}:{labour_table}"
units = "$"

line_plot(
    productivity,
    dropna=True,
    title=title,
    ylabel=units,
    rfooter=rfooter,
    lfooter=lfooter,
    show=SHOW,
)

growth = calc_growth(productivity)
plot_growth_finalise(
    *growth,
    title=f"Growth in {title}",
    rfooter=rfooter,
    lfooter=lfooter,
    show=SHOW,
)

### Headline

In [8]:
series_type = "Seasonally Adjusted"
not_unit = "Percent"
table = "1"
data = abs_labour[table]

rows = meta_labour[
    (meta_labour[metacol.table] == table)
    & (meta_labour[metacol.stype] == series_type)
    & (meta_labour[metacol.unit] != not_unit)
]

for index, row in rows.iterrows():
    series_id, units, title = row[metacol.id], row[metacol.unit], row[metacol.did]
    plotable, units = recalibrate_series(data[series_id], units)
    plotable.name = f"{series_type.capitalize()} series"

    plot_covid_recovery(
        plotable,
        title=(
            title.replace(" ;  Australia ;  Total all industries ;", "")
            .replace(" per Labour Account", "\nper Labour Account")
            .replace(" ; ", " - ")
            .replace("; ", ": ")
        ),
        tags="covid",
        ylabel=units,
        rfooter=f"{source} T{table}",
        lfooter="Australia. All industries. ",
        show=SHOW,
    )

## Finished

In [9]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sun Feb 18 2024 21:13:36

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.21.0

matplotlib: 3.8.3
sys       : 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
pandas    : 2.2.0

Watermark: 2.4.3



In [10]:
print("Finished")

Finished
